# Database from
https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data?select=full_df.csv

In [1]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from imblearn.under_sampling import RandomUnderSampler

from helpers.help import *
from helpers.helptf import *
from sklearn.utils import resample

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input

# Getting rethinophaty data

In [2]:
# load dataset
df = pd.read_csv('hr-dataset/full_df.csv')

# get the diagnostic of hypertensive retinopathy
ds_hr = df[df['Left-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]

# get the diagnostic of diabetic retinopathy
ds_dr = df[df['Left-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]

# get the diagnostic of normal fundus
ds_normal = df[df['Left-Diagnostic Keywords'] == 'normal fundus']

# Specific dataframe
df_hr = ds_hr[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_dr = ds_dr[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_normal = ds_normal[['Left-Diagnostic Keywords', 'Left-Fundus']]




# Droping class
df_hr = df_hr.drop('Left-Diagnostic Keywords', axis=1)
df_dr = df_dr.drop('Left-Diagnostic Keywords', axis=1)
df_normal = df_normal.drop('Left-Diagnostic Keywords', axis=1)



print(df_hr.shape[0])
print(df_dr.shape[0])
print(df_normal.shape[0])

191
85
2796


# Solving the undersampling of HR 

In [3]:
df_hr_downsampled = resample(df_hr, replace=False, n_samples=85, random_state=10)
df_dr_downsampled = resample(df_dr, replace=False, n_samples=85, random_state=10)
df_normal_downsampled = resample(df_normal, replace=False, n_samples=85, random_state=10)

print(df_hr_downsampled.shape[0])
print(df_dr_downsampled.shape[0])
print(df_normal_downsampled.shape[0])


85
85
85


# Class transformation

In [4]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'hr-dataset/preprocessed_images')

# 0 - Normal
# 1 - Diabetic Rethinopaty
# 2 - Hipertensive Rethinopaty

# roam Hipertensive rethinopaty
array = []
for index, row in df_hr_downsampled.iterrows():
    detailPath = os.path.join(path,row['Left-Fundus'])
    array.append([detailPath,2])
    
    
# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

85

# Get the data ready

In [5]:
X,y=dataset[::,0],dataset[::,1]
y = y.astype(int)

#One hot encode the labels
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 20% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.2*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 20% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
print(f"Samples in Training set: {x_train.shape[0]}")
print(f"Samples in Validation set: {x_val.shape[0]}")
print(f"Samples in Test set: {x_test.shape[0]}")

Samples in Training set: 54
Samples in Validation set: 14
Samples in Test set: 17


In [7]:
# Check if imbalance
for i in [y_train, y_test, y_val]:
    print(np.unique(i, return_counts = True, axis = 0))

(array([[0., 0., 1.]]), array([54]))
(array([[0., 0., 1.]]), array([17]))
(array([[0., 0., 1.]]), array([14]))


# Prepares Data for the model

In [8]:
val_dataset=build_dataset(x_val,y_val,repeat=False,batch=256)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=256)

BATCH_SIZE=8
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_train,y_train,batch=BATCH_SIZE)

# input shape for the model
input_shape=train_dataset.element_spec[0].shape[1:]


print(train_dataset)
print(val_dataset)
print(test_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>


# Load model

In [ ]:
dr_model = load_model('model/model_baseline.keras')

"""
print(dr_model.layers)

# Define the input tensor explicitly
input_tensor = Input(shape=input_shape)  # Match the original input shape

# Create the feature extractor
feature_extractor = Model(inputs=input_tensor, outputs=dr_model.layers[-7].output)


for layer in feature_extractor.layers:
    layer.trainable = False  # Freeze feature extractor

print(input_shape)
print(dr_model.layers)
"""
dr_model.summary()

<bound method Model.summary of <Sequential name=sequential_1, built=True>>

# Add the new layer

In [17]:
# Add new dense layers for your specific classification task

"""
transfer_model = Sequential([
    feature_extractor,               # Use the feature extractor
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax')  # classes: {"no_rethinopathy", "dr","hr"}
])"""


# classes: {"no_rethinopathy", "dr","hr"}
transfer_model = prep_translearn(model=dr_model, top_layers_to_cut=7, out_dim=3, learning_rate=0.001) 

ValueError: Input 0 of layer "batch_normalization_4" is incompatible with the layer: expected axis -1 of input shape to have value 32, but received input with shape (None, 64, 64, 3)

# Prepare transfer model

In [ ]:
transfer_model.compile(
        loss = "categorical_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )

transfer_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_375 (Functional)     │ (None, 6, 6, 128)      │        94,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 6, 6, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 6, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 6, 6, 3)        │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,963 (503.76 KB)

 Trainable params: 128,003 (500.01 KB)

 Non-trainable params: 960 (3.75 KB)

# Train model

In [ ]:
EPOCHS = 20
STEPS_PER_EPOCH = 20
transfer_model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=EPOCHS,
          validation_data=val_dataset,validation_steps=None)

Epoch 1/20


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m6254873776\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(None, 64, 64, 3), dtype=float32)\n  • training=True\n  • mask=None'